<a href="https://colab.research.google.com/github/Arjun-RY/Projects/blob/task/Task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [92]:
df1 = pd.read_csv('/content/drive/MyDrive/Task1/takehome_users.csv',encoding="ISO-8859-1")
df2 = pd.read_csv('/content/drive/MyDrive/Task1/takehome_user_engagement.csv',encoding="ISO-8859-1")
col1 = list(df1.columns.values)
col2 = list(df2.columns.values)
print(col1,col2)
df2=df2[['user_id','time_stamp', 'visited']]
df2.columns = ['object_id','time_stamp', 'visited']
col2 = list(df2.columns.values)
print(col2)

['object_id', 'creation_time', 'name', 'email', 'creation_source', 'last_session_creation_time', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id'] ['time_stamp', 'user_id', 'visited']
['object_id', 'time_stamp', 'visited']


In [ ]:
for col in df1:
  print(df1[col].unique())
print(df1.nunique())
display(df1.isnull().sum())

In [95]:
df2['object_id'].nunique()

8823

In [96]:
df2['time_stamp'] = pd.to_datetime(df2['time_stamp'])
df2.set_index("time_stamp",inplace=True)
ddf2=df2.groupby('object_id').resample("W").sum()
ddf2.drop(['object_id'],axis=1,inplace=True)
ddf2.reset_index("time_stamp",inplace=True)
ddf3=ddf2.groupby('object_id').agg({'visited':'max'})
ddf3.reset_index("object_id",inplace=True)
adopted_user=[]
for row in ddf3['visited']:
  if row>=3:
    adopted_user.append(1)
  else:
    adopted_user.append(0)
ddf3['adopted_user']=adopted_user
ddf3

,object_id,visited,adopted_user
0,1,1,0
1,2,3,1
2,3,1,0
3,4,1,0
4,5,1,0
...,...,...,...
8818,11996,1,0
8819,11997,1,0
8820,11998,1,0
8821,11999,1,0


In [98]:
print("Count of adopted users ->", ddf3[(ddf3['adopted_user']==1)]['object_id'].count())
#ddf3[(ddf3['adopted_user']==1)]

Count of adopted users -> 1445


In [ ]:
df2[(df2['object_id']==11965)] # to check whether the adopted user has logged in 3 times within a week

In [105]:
df=pd.merge(df1,ddf3,on='object_id',how='outer')

In [106]:
df["adopted_user"] = df["adopted_user"].fillna(0)
print("Count of adopted users ->", df[(df['adopted_user']==1)]['object_id'].count())

Count of adopted users -> 1445


In [ ]:
print(df.nunique())

In [ ]:
for col in df:
  print(df[col].unique())

In [109]:
display(df.isnull().sum())
df.drop(['last_session_creation_time','visited','invited_by_user_id'],axis=1,inplace=True)

object_id                        0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
visited                       3177
adopted_user                     0
dtype: int64

In [110]:
print("Count of adopted users ->", df[(df['adopted_user']==0)]['object_id'].count())

Count of adopted users -> 10555


In [111]:
df.corr()

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted_user
object_id,1.000000,-0.032370,-0.022040,0.004110,0.007481
opted_in_to_mailing_list,-0.032370,1.000000,0.483529,0.003432,0.010339
enabled_for_marketing_drip,-0.022040,0.483529,1.000000,0.009275,0.005901
org_id,0.004110,0.003432,0.009275,1.000000,0.060683
adopted_user,0.007481,0.010339,0.005901,0.060683,1.000000


###Summary:

>    


####After working on takehome_user_engagement csv file we get only 1445 adopted users out of 12000 users.
####After merging the two dataframes , we drop the last_session_creation_time column since data in that column are unclear and contains null values which cannot be replaced. We drop the invited_by_user_id column because it contains nearly 5500 null values which cannot be replaced and drop the visited column because the values inside the column are used to create the adopted user column and it naturally has ***good correlation with adopted user***.


####From the above correlation matrix of the dataframe we can infer that the factors (object_id, opted_in_to_mailing_list, enabled_for_marketing_drip, org_id) has bad correlation with adopted user; since their correlation values are very much less than 0.6 . Hence, ***we cannot predict future user adoption using these factors.***
